In [1]:
import pickle
import pandas as pd
from glob import glob
from pandas import Index
import sys
sys._enablelegacywindowsfsencoding()
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def signal_process(table,event_var='PM25_Marseille_Longchamp',
                   event_threshold = 40, duration = 12,var_name_list='air pollution'):
    """Find the first spike in pm25 and get a window of length duration for the variable (non overlap in )
    table: pandas table
        must include datetime 
    event_var: str
        the dominant sequence that preceding other signals
    event_threshold:  int
        the cirical threshold of event_var 
    duration:  int
        the length of observation
    var_name_list: list
        the variable list worth seeing
    """
    
    origin_array=np.where(table[event_var]>event_threshold)[0]
    
    shrink_index_list=[]
    shrink_index_list.append(origin_array[0])
    for idx in range(1,len(origin_array)):
        if origin_array[idx]>shrink_index_list[-1]+duration:
            shrink_index_list.append(origin_array[idx])
        
    
    bind_list = []
    for start_index in shrink_index_list:
        this_sequence = table.loc[range(start_index-2,start_index+duration),
                                  var_name_list]
        start_datetime = table.datetime[start_index]
        tuples = list(zip([start_datetime]*(duration+2),range(-2,duration)))

        this_sequence.index = pd.MultiIndex.from_tuples(tuples,names=['event_date_index','hour_index'])
    #         this_sequence = (this_sequence- this_sequence.mean())/this_sequence.std()

        bind_list.append(this_sequence)
    new_table=pd.concat(bind_list)     
    
    return new_table

In [3]:
def get_table(path="..\\stage\\FR-J\\*"):
    df_list =[]
    for filepath in glob(path):
        with open(filepath,'br') as f:
            this_df = pickle.load(f).reset_index()
            col_name = list(this_df.columns)
            col_name[2]=col_name[1]+"_topic"
            this_df.columns =Index(col_name,dtype='object')
            df_list.append(this_df)
    merged_t = df_list[0]
    for i in range(1,len(df_list)):
        merged_t=merged_t.merge(df_list[i],how='outer',on='date')
    return merged_t

In [4]:
parca=pd.read_csv('../stage/difficultés respiratoires_asthme_dyspnée_respiration_PM25_Marseille_Longchamp.csv')

In [5]:
parca['date']=parca.date.apply(lambda string: datetime.strptime(string,'%Y-%m-%d %H:%M:%S') )

In [8]:
parca_merged=parca.merge(get_table(path='..\\stage\\FR-U\\*'),on='date')

In [9]:
parca_merged.head()

,Unnamed: 0,datetime,PM25_Marseille_Longchamp,date,difficultés respiratoires,asthme,dyspnée,respiration,isPartial_x,banane,banane_topic,isPartial_x,la diarrhée,la diarrhée_topic,isPartial_y,pollution de l’air,pollution de l’air_topic,isPartial_y
0,0,2017-08-31 00:00:00,8.0,2017-08-31 00:00:00,0,13,0,0,False,46,54,False,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2017-08-31 01:00:00,9.0,2017-08-31 01:00:00,0,0,0,79,False,23,35,False,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2017-08-31 02:00:00,8.0,2017-08-31 02:00:00,0,0,23,46,False,27,41,False,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2017-08-31 03:00:00,7.0,2017-08-31 03:00:00,0,0,0,68,False,10,20,False,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2017-08-31 04:00:00,8.0,2017-08-31 04:00:00,0,0,0,26,False,42,52,False,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
parca_merged=parca_merged.drop(columns=['Unnamed: 0','date','isPartial_x','isPartial_x','isPartial_y'])

In [11]:
my_list =list(parca_merged.columns)
my_list.remove('datetime')

In [12]:
my_list

['PM25_Marseille_Longchamp',
 'difficultés respiratoires',
 'asthme',
 'dyspnée',
 'respiration',
 'banane',
 'banane_topic',
 'la diarrhée',
 'la diarrhée_topic',
 'pollution de l’air',
 'pollution de l’air_topic']

In [13]:
parca_signal=signal_process(parca_merged,event_var='PM25_Marseille_Longchamp',duration=12,event_threshold=35,var_name_list=my_list)

In [74]:
# parca_signal

In [75]:
parca_signal.to_csv("../output/air_parca_threshold_35_colsum_duration_12H.csv")

In [ ]:
# parif 

In [14]:
parif_com=get_table(path='..\\stage\\FR-J\\*')

In [15]:
parif_table = pd.read_csv("../stage/parif_asthme_dyspnée_respiration_pollution de l’air_Masque_Purificateur d'air_PA04C_PM10.csv")

In [16]:
parif_table['date']=parca.datetime.apply(lambda string: datetime.strptime(string,'%Y-%m-%d %H:%M:%S'))
parif_table['datetime']=parif_table['date']

In [17]:
parif_merged=parif_table.merge(parif_com,on='date')

In [18]:
parif_merged=parif_merged[['PA04C_PM25','datetime', 'date', 'asthme', 'asthme topic', 'dyspnée',
       'dyspnée topic', 'respiration', 'respiration topic',
       'pollution de l’air', 'pollution de l’air topic', 
       'Masque topic', 'la diarrhée',
       'la diarrhée_topic',
       'banane', 'banane_topic',]]

In [19]:
parif_list = list(parif_merged.columns)
parif_list.remove('datetime')
parif_list.remove('date')

In [20]:
parif_signal=signal_process(parif_merged,event_var='PA04C_PM25',duration=12,event_threshold=35,var_name_list=parif_list)

In [21]:
parif_signal.to_csv("../output/air_parif_threshold_35_colsum_duration_12H.csv")

In [ ]:
# limoges

In [22]:
Limoges_table = pd.read_csv("../stage/limoges.csv").drop(columns=['Year','Month','Day','UTC Hour'])
Limoges_table.columns=Index(['PM25_Limoges', 'PM10_Limoges', 'datetime'], dtype='object')

In [23]:
def conv(datestring):
    try :
        val = datetime.strptime(datestring ,'%Y-%m-%d %H:%M:%S')
    except TypeError:
        val = datestring
    return val

In [24]:
Limoges_table['datetime'] = Limoges_table['datetime'].apply(conv)

In [25]:
Limoge_com=get_table(path="..\\stage\\FR-B\\*")

In [26]:
Limoge_com=Limoge_com.drop(columns=['isPartial_x','isPartial_y','isPartial'])

In [27]:
Limoges_merged=Limoges_table.merge(Limoge_com,left_on='datetime',right_on='date')

In [28]:
Limoges_list =list(Limoges_merged.columns)
Limoges_list.remove("date")
Limoges_list.remove("datetime")

In [29]:
Limoges_signal=signal_process(Limoges_merged,event_var='PM25_Limoges',duration=12,event_threshold=35,var_name_list=Limoges_list)

In [101]:
Limoges_signal.to_csv("../output/air_limoges_threshold_35_colsum_duration_12H.csv")

In [44]:
Limoges_merged.drop_duplicates(inplace=True)

In [45]:
parca_merged.drop_duplicates(inplace=True)

In [46]:
parif_merged.drop_duplicates(inplace=True)

In [47]:
Limoges_merged.to_csv("../stage/final_limoges.csv")

In [48]:
parca_merged.to_csv("../stage/final_marseille.csv")

In [49]:
parif_merged.to_csv("../stage/final_paris.csv")